In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import os 
import PIL
import time

from IPython import display

## Load Data

In [ ]:
dataset_name = 'diabetic_retinopathy_detection/btgraham-300'
subdataset_name = 'btgraham-300'
print(dataset_name)

In [ ]:
trainning_dir = 'c1_'


## Global Variables

In [ ]:
BATCH_SIZE = 256
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
IMG_SIZE = 64

### Load dataset from memory

In [ ]:
new_dataset, info= tfds.load(dataset_name,split=[tfds.Split.TRAIN, tfds.Split.TEST], download=True, with_info=True)

dataset = new_dataset[0]
dataset = dataset.concatenate(new_dataset[1])

# print(type(dataset))


In [ ]:
print(dataset.__len__())

In [ ]:

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x: (resize_and_rescale(x['image'])), 
              num_parallel_calls=AUTOTUNE)
              
  return ds.prefetch(buffer_size=AUTOTUNE)

## Create Models

### Create Generator

In [ ]:
def make_generator_model():
  aKernelSize = 5
  aStrides = 2
  z_dim = 100

  layersInfo = [(4, 1024), (8, 512), (16,256), (32, 128), (64, 3)]
  
  layer_0 = (4, 1024)
  model = tf.keras.Sequential()
  model.add(layers.Dense(layer_0[0] ** 2 * layer_0[1], use_bias=False, input_shape=(z_dim,), name='gen_1'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  
  model.add(layers.Reshape((layer_0[0], layer_0[0], layer_0[1])))
  print(model.output_shape)
  assert model.output_shape == (None, layer_0[0], layer_0[0], layer_0[1])  # Note: None is the batch size

  layer_1 = layersInfo[1]
  model.add(layers.Conv2DTranspose(layer_1[1], kernel_size=aKernelSize, strides=aStrides, padding='same', use_bias=False, name='gen_2'))
  print(model.output_shape)
  assert model.output_shape == (None, layer_1[0], layer_1[0], layer_1[1])
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  layer_2 = layersInfo[2]
  model.add(layers.Conv2DTranspose(layer_2[1], kernel_size=aKernelSize, strides=aStrides, padding='same', use_bias=False, name='gen_3'))
  assert model.output_shape == (None, layer_2[0], layer_2[0], layer_2[1])
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  layer_3 = layersInfo[3]
  model.add(layers.Conv2DTranspose(layer_3[1], kernel_size=aKernelSize, strides=aStrides, padding='same', use_bias=False, name='gen_4'))
  assert model.output_shape == (None, layer_3[0], layer_3[0], layer_3[1])
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU())

  layer_4 = layersInfo[4]
  model.add(layers.Conv2DTranspose(layer_4[1], kernel_size=aKernelSize, strides=aStrides, padding='same', use_bias=False, activation='tanh', name='gen_5'))
  assert model.output_shape == (None, layer_4[0], layer_4[0], layer_4[1])

  return model

### Instanciate the generator model

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)
print(generated_image.shape)
plt.imshow(generated_image[0, :, :, 0], cmap='gray')

### Create Discriminator

In [ ]:
def make_discriminator_model():
  aKernelSize = 5
  aStrides = 2
  dropOutRatio = 0.3

  layersInfo = [(32, 64), (16,128), (8, 256), (4, 512)]
  
  model = tf.keras.Sequential()
  
  layer_0 = layersInfo[0]
  model.add(layers.Conv2D(layer_0[1], kernel_size=aKernelSize, strides=aStrides, padding='same', input_shape=[64, 64, 3], name='dis_1'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(dropOutRatio))

  layer_1 = layersInfo[1]
  model.add(layers.Conv2D(layer_1[1], kernel_size=aKernelSize, strides=aStrides, padding='same', name='dis_2'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(dropOutRatio))

  layer_2 = layersInfo[2]
  model.add(layers.Conv2D(layer_2[1], kernel_size=aKernelSize, strides=aStrides, padding='same', name='dis_3'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(dropOutRatio))

  layer_3 = layersInfo[3]
  model.add(layers.Conv2D(layer_3[1], kernel_size=aKernelSize, strides=aStrides, padding='same', name='dis_4'))
  model.add(layers.LeakyReLU())
  model.add(layers.Dropout(dropOutRatio))

  model.add(layers.Flatten())
  model.add(layers.Dense(1)
)

  return model

### Instanciate discriminator model

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)


In [ ]:
print(discriminator.layers)

## Define Cross Entropy, Discriminator Loss and generator Loss

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
learing_rate = 1e-4
generator_optimizer = tf.keras.optimizers.Adam(learing_rate)
discriminator_optimizer = tf.keras.optimizers.Adam(learing_rate)

### Generate Checkpoint variable

In [ ]:
checkpoint_dir = './' + trainning_dir + 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

### Load checkpoint variables

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Generate Seed

In [ ]:

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

## Prepare trainning functions

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def generate_and_save_images(model, epoch, test_input, cmap=None):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap=cmap)
      plt.axis('off')

  plt.savefig(trainning_dir + 'multi_image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

#generate_and_save_images(generator,50,seed)

In [ ]:
def generate_one_save_images(model, epoch, test_input, cmap=None):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure()
  plt.imshow(predictions[0, :, :, 0], cmap=cmap)

  plt.savefig(trainning_dir + 'one_image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
def train(dataset, epochs, initial_epochs=0):
  epoch = initial_epochs
  while epoch < epochs:
  #for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    display.clear_output(wait=True)
    # Save multiple images
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)
    # Save only one image
    generate_one_save_images(generator,
                             epoch + 1,
                             seed)
    # Save the model every 10 epochs
    #if (epoch + 1) % 10 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    epoch += 1
  # Generate after the final epoch

# Train the model

In [ ]:
train_dataset = prepare(dataset).batch(BATCH_SIZE)
last_epoch = 50
train(train_dataset, 60, initial_epochs=last_epoch)

In [ ]:
generate_image = generator(noise, training=False)
plt.imshow(generate_image[0, :, :, 0], cmap=None)